In [3]:
from typing import Any, Optional, Union, List
from pydantic import BaseModel, PositiveInt, PositiveFloat
from datetime import datetime
import os
from sortedcontainers import SortedList
from enum import Enum, auto
    
    

In [11]:
class OrderIdGenerator:
    """
    A class that generates unique order IDs.

    Attributes:
        filepath (str): The file path to store the current state of the ID counter.
        id_counter (int): The current value of the ID counter.
        iterations_since_last_save (int): The number of iterations since the last state save.
        save_frequency (int): The frequency at which the state should be saved.

    Methods:
        __iter__(): Returns the iterator object itself.
        __next__(): Returns the next unique order ID.
        save_state(): Saves the current state of the ID counter to a file.
        load_state(): Loads the previous state of the ID counter from a file.
        reset_state(): Resets the ID counter to 0 and saves the state.
    """

    def __init__(self):
        self.filepath = os.path.join(os.getcwd(), "current_state.txt")
        self.id_counter = self.load_state()
        self.iterations_since_last_save = 0
        self.save_frequency = 100

    def __iter__(self):
        return self

    def __next__(self):
        current_id = self.id_counter
        self.id_counter += 1
        self.iterations_since_last_save += 1
        if self.iterations_since_last_save >= self.save_frequency:
            self.save_state()
            self.iterations_since_last_save = 0

        return current_id

    def save_state(self) -> None:
        """
        Saves the current state of the ID counter to a file.
        """
        with open(self.filepath, "w", encoding="utf-8") as f:
            f.write(str(self.id_counter))

    def load_state(self) -> None:
        """
        Loads the previous state of the ID counter from a file.
        """
        if not os.path.exists(self.filepath):
            return 0
        with open(self.filepath, "r", encoding="utf-8") as f:
            saved_state = f.read()
            return int(saved_state)

    def reset_state(self) -> None:
        """
        Resets the state of the object by setting the id_counter to 0 and saving the state.
        """
        self.id_counter = 0
        self.save_state()

In [18]:
class OrderStatus(Enum):
    CREATED = auto()
    PARTIALLY_FILLED = auto()
    FILLED = auto()
    MODIFIED = auto()
    CANCELLED = auto()
    RESTORED = auto()
    EXPIRED = auto()

    @property
    def is_active(self):
        return self in {
            OrderStatus.CREATED,
            OrderStatus.PARTIALLY_FILLED,
            OrderStatus.MODIFIED,
            OrderStatus.RESTORED,
        }


class OrderType(Enum):
    ASK = "ask"
    BID = "bid"


class Order(BaseModel):
    id: PositiveInt
    order_type: OrderType
    price: PositiveFloat
    volume: PositiveInt
    owner_id: PositiveInt
    status: Optional[OrderStatus] = None
    created: Optional[datetime] = None
    updated: Optional[datetime] = None
    listed: Optional[datetime] = None

    def __init__(self, **data):
        # logging here
        super().__init__(**data)
        if self.status is None:
            self.status = OrderStatus.CREATED
        now = datetime.now()
        if self.created is None:
            self.created = now
        if self.updated is None:
            self.updated = now

    def __setattr__(self, name: str, value: Any) -> None:
        if name != "updated":
            super().__setattr__("updated", datetime.now())
        # logging here
        return super().__setattr__(name, value)

    def __lt__(self, other: Union[int, float, "Order"]):
        if isinstance(other, (int, float)):
            return self.price < other
        if isinstance(other, Order):
            return self.price < other.price
        return NotImplemented

    def __gt__(self, other: Union[int, float, "Order"]):
        if isinstance(other, (int, float)):
            return self.price > other
        if isinstance(other, Order):
            return self.price > other.price
        return NotImplemented

    def __le__(self, other: Union[int, float, "Order"]):
        if isinstance(other, (int, float)):
            return self.price <= other
        if isinstance(other, Order):
            return self.price <= other.price
        return NotImplemented

    def __ge__(self, other: Union[int, float, "Order"]):
        if isinstance(other, (int, float)):
            return self.price >= other
        if isinstance(other, Order):
            return self.price >= other.price
        return NotImplemented


class OrderList:
    """
    Order List is a collection of orders. It is used to store and manage orders.
    Active orders are stored in a sorted list with search complexity of O(log n) and insertion complexity of O(n).
    All orders are indexed by their id for O(1) access.
    Active orders can be reached through bisect operations.
    """

    def __init__(
        self,
        order_type: OrderType,
        order_list: Optional[List[Order]] = None,
    ) -> None:
        """
        Initializes a collection of orders of a specified type. All orders within the collection must share the same order type.

        :param order_type: The type of orders to be stored in the list, e.g., 'ask' or 'bid'. The specific types are defined in the OrderType enum.
        :type order_type: OrderType
        :param order_list: Initial list of orders to be added to the collection, defaults to None. Each order in the list is added to the collection using the 'add' method logic.
        :type order_list: Optional[List[Order]], optional
        """
        self.__order_list = SortedList()
        self.__ids = {}
        self.otype = order_type
        if order_list:
            for order in order_list:
                self.add(order)

    def get(self, order_id: int) -> Optional[Order]:
        """
        Gets an order by its ID.

        :param order_id: The ID of the order to retrieve.
        :type order_id: int
        :return: The order with the specified ID, if found; otherwise, None.
        :rtype: Optional[Order]
        """
        return self.__ids.get(order_id)

    def add(self, order: Order, tolist: Union[bool, str] = "auto") -> None:
        """
        Adds an order to the collection. The order is only added to the sorted list if it meets the criteria defined by the 'tolist' parameter. By default ('auto'), active orders are added to the sorted list.

        :param order: The order to be added to the collection.
        :type order: Order
        :param tolist: Specifies how the order should be added to the list. 'auto' adds active orders automatically; True or 'y'/'yes' always adds; False or 'n'/'no' never adds. Defaults to 'auto'.
        :type tolist: Union[bool, str], optional
        :raises ValueError: If the order's type does not match the collection's type.
        :raises ValueError: If 'tolist' is given an invalid value.
        """
        if order.order_type != self.otype:
            raise ValueError("Order type must be the same as the list type")
        if tolist == "auto":
            if order.status.is_active:
                if order.listed is None:
                    order.listed = datetime.now()
                self.__order_list.add(order)
        elif tolist is True or tolist in ["y", "yes"]:
            self.__order_list.add(order)
        elif tolist is False or tolist in ["n", "no"]:
            pass
        else:
            raise ValueError("Invalid value for tolist")
        self.__ids[order.id] = order

    def bisect_left(
        self, order: Union[Order, float], include_right: bool = True
    ) -> List[Order]:
        """
        Performs a bisect left operation on the sorted list to find the position to insert 'order' or 'price'.
        Returns a segment of the list based on the bisect position.

        :param order: Either an Order object or a float price value to perform the bisect operation.
        :type order: Union[Order, float]
        :param include_right: If True, returns the segment of the list from the bisect position to the end (inclusive of the position).
                              If False, returns the segment up to the bisect position (exclusive).
                              Defaults to True.
        :type include_right: bool, optional
        :return: A list of Orders filtered based on the bisect operation.
        :rtype: List[Order]
        """
        cid = self.__order_list.bisect_left(order)
        if include_right:
            return self.__order_list[cid:]
        return self.__order_list[:cid]

    def bisect_right(
        self, order: Union[Order, float], include_left: bool = True
    ) -> List[Order]:
        """
        Performs a bisect right operation on the sorted list to find the position to insert 'order' or 'price'.
        Returns a segment of the list based on the bisect position.

        :param order: Either an Order object or a float price value to perform the bisect operation.
        :type order: Union[Order, float]
        :param include_left: If True, returns the segment of the list up to the bisect position (inclusive).
                             If False, returns the segment of the list from the bisect position to the end (exclusive).
                             Defaults to True.
        :type include_left: bool, optional
        :return: A list of Orders filtered based on the bisect operation.
        :rtype: List[Order]
        """
        cid = self.__order_list.bisect_right(order)
        if include_left:
            return self.__order_list[:cid]
        return self.__order_list[cid:]

    def unlist(self, order: Union[Order, int], order_status: OrderStatus) -> None:
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")
        if not order.status.is_active:
            raise ValueError("Order is not active.")
        if order_status.is_active:
            raise ValueError("Order cannot be unlisted with active status.")
        self.__order_list.remove(order)
        order.status = order_status

    def relist(self, order: Union[Order, int], order_status: OrderStatus = OrderStatus.RESTORED) -> None:
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")
        if not order_status.is_active:
            raise ValueError("Active order cannot have non-active status")
        order.status = order_status
        order.listed = datetime.now()
        self.__order_list.add(order)

    def remove(self, order: Union[Order, int]) -> None:
        """
        Remove an order from the collection.

        :param order: order to be removed, it can be either an order or an order id
        :type order: Union[Order, int]
        :raises ValueError: if provided order is invalid type
        :raises ValueError: if order is not found
        """
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")

        if order.status.is_active:
            self.__order_list.remove(order)
        del self.__ids[order.id]

    def expire(self, order: Union[Order, int]) -> None:
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")
        if not order.status.is_active:
            raise ValueError("Order is not active.")
        self.__order_list.remove(order)
        order.status = OrderStatus.EXPIRED

    def cancel(self, order: Union[Order, int]) -> None:
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")
        if not order.status.is_active:
            raise ValueError("Order is not active.")
        self.__order_list.remove(order)
        order.status = OrderStatus.CANCELLED

    def fill(self, order: Union[Order, int], volume: int) -> None:
        if volume <= 0:
            raise ValueError("Volume must be positive")
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")
        if not order.status.is_active:
            raise ValueError("Order is not active.")
        if volume > order.volume:
            raise ValueError("Volume is greater than order volume")
        order.volume -= volume
        if order.volume == 0:
            self.unlist(order, OrderStatus.FILLED)
        else:
            order.status = OrderStatus.PARTIALLY_FILLED

    def modify(
        self,
        order: Union[Order, int],
        price: Optional[float] = None,
        volume: Optional[int] = None,
        relist: bool = True,
    ) -> None:
        if any([price<=0, volume<=0]):
            raise ValueError("Price and volume must be positive")
        if isinstance(order, int):
            order = self.__ids.get(order)
            if order is None:
                raise ValueError("Provided order ID not found")
        elif not isinstance(order, Order):
            raise ValueError("Invalid order type")

        if relist:
            if order.status.is_active:
                self.__order_list.remove(order)
            else:
                print("Warning: Order is not active, it will not be relisted")
                relist = False
        order = self.__ids[order.id]
        if price is not None:
            order.price = price
        if volume is not None:
            order.volume = volume
        if relist:
            order.status = OrderStatus.MODIFIED
            self.__order_list.add(order)

    def clear(self) -> None:
        """
        Clears the collection of all orders.
        """
        self.__order_list.clear()
        self.__ids.clear()

    def __getitem__(self, order_id):
        return self.__ids[order_id]

    def __iter__(self):
        return iter(self.__order_list)

    def __len__(self):
        return len(self.__ids)

    def __repr__(self):
        return str(self.__order_list)

# simple tests

In [5]:
import random

### creation

In [6]:
orders = []
for i in range(random.randint(10, 25)):
    orders.append(
        Order(
            id=i+1,
            order_type=OrderType.ASK,
            price=random.uniform(95, 110),
            volume=random.randint(10, 70),
            owner_id=random.randint(1, 10),
        )
    )
orders[:5]

[Order(id=1, order_type=<OrderType.ASK: 'ask'>, price=108.15772034491731, volume=40, owner_id=2, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 25, 17, 35, 33, 162363), updated=datetime.datetime(2024, 2, 25, 17, 35, 33, 162363), listed=None),
 Order(id=2, order_type=<OrderType.ASK: 'ask'>, price=100.70032130314736, volume=24, owner_id=9, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 25, 17, 35, 33, 162363), updated=datetime.datetime(2024, 2, 25, 17, 35, 33, 162363), listed=None),
 Order(id=3, order_type=<OrderType.ASK: 'ask'>, price=99.82702453206373, volume=58, owner_id=9, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 25, 17, 35, 33, 162363), updated=datetime.datetime(2024, 2, 25, 17, 35, 33, 162363), listed=None),
 Order(id=4, order_type=<OrderType.ASK: 'ask'>, price=101.98836571267226, volume=13, owner_id=9, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 25, 17, 35, 33, 162363), updated=datetime.da

In [7]:
order_list = OrderList(order_type=OrderType.ASK, order_list=orders)
order_list

SortedList([Order(id=15, order_type=<OrderType.ASK: 'ask'>, price=95.17734461862833, volume=55, owner_id=6, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 25, 17, 35, 33, 162363), updated=datetime.datetime(2024, 2, 25, 17, 35, 33, 179041), listed=datetime.datetime(2024, 2, 25, 17, 35, 33, 179041)), Order(id=14, order_type=<OrderType.ASK: 'ask'>, price=95.71232072962259, volume=35, owner_id=4, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 25, 17, 35, 33, 162363), updated=datetime.datetime(2024, 2, 25, 17, 35, 33, 179041), listed=datetime.datetime(2024, 2, 25, 17, 35, 33, 179041)), Order(id=13, order_type=<OrderType.ASK: 'ask'>, price=96.94017979162851, volume=51, owner_id=3, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 25, 17, 35, 33, 162363), updated=datetime.datetime(2024, 2, 25, 17, 35, 33, 179041), listed=datetime.datetime(2024, 2, 25, 17, 35, 33, 179041)), Order(id=7, order_type=<OrderType.ASK: 'ask'>, price=97.02315

### wrong order

In [8]:
wrong_order = Order(
    id=100,
    order_type=OrderType.BID,
    price=100,
    volume=100,
    owner_id=1,
)
order_list.add(wrong_order)
# error

ValueError: Order type must be the same as the list type

### remove, unlist, relist

In [ ]:
random_order = random.choice(orders)

print(random_order.id)
order_list.remove(random_order)
order_list[random_order.id]
# error

18


KeyError: 18

In [ ]:
random_order = random.choice(orders)

print(random_order.id)
order_list.unlist(random_order, OrderStatus.FILLED)
print(order_list[random_order.id])
# inactive
order_list.relist(random_order)
print(order_list[random_order.id])
# active
order_list.expire(random_order)
print(order_list[random_order.id])
# expired
order_list.relist(random_order)
order_list.cancel(random_order)
print(order_list[random_order.id])
order_list.cancel(random_order)
# error order is already canceled


6
id=6 order_type=<OrderType.ASK: 'ask'> price=101.70131585168942 volume=32 owner_id=4 status=<OrderStatus.FILLED: 3> created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257) updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 970819) listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)
id=6 order_type=<OrderType.ASK: 'ask'> price=101.70131585168942 volume=32 owner_id=4 status=<OrderStatus.RESTORED: 6> created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257) updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 971817) listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 971817)
id=6 order_type=<OrderType.ASK: 'ask'> price=101.70131585168942 volume=32 owner_id=4 status=<OrderStatus.EXPIRED: 7> created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257) updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 971817) listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 971817)
id=6 order_type=<OrderType.ASK: 'ask'> price=101.70131585168942 volume=32 owner_id=4 status=<OrderStatus.CANCELLED: 5> 

ValueError: Order is not active.

In [ ]:
order_list.relist(random_order)
order_list.unlist(random_order, OrderStatus.PARTIALLY_FILLED)
# error

ValueError: Order cannot be unlisted with active status.

### filling 

In [ ]:
random_order = random.choice(orders)
print(random_order)
order_list.fill(random_order, 10)
print(order_list[random_order.id])

id=16 order_type=<OrderType.ASK: 'ask'> price=96.4238155325062 volume=30 owner_id=3 status=<OrderStatus.CREATED: 1> created=datetime.datetime(2024, 2, 24, 20, 20, 15, 188255) updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008) listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)
id=16 order_type=<OrderType.ASK: 'ask'> price=96.4238155325062 volume=20 owner_id=3 status=<OrderStatus.PARTIALLY_FILLED: 2> created=datetime.datetime(2024, 2, 24, 20, 20, 15, 188255) updated=datetime.datetime(2024, 2, 24, 20, 20, 16, 842710) listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)


In [ ]:
order_list.fill(random_order, random_order.volume + 10)
# error

ValueError: Volume is greater than order volume

In [ ]:
order_list.fill(random_order, -11)
# error

ValueError: Volume must be positive

In [ ]:
random_order = random.choice(orders)
order_list.fill(random_order, random_order.volume)
# filled order will be unlisted
order_list[random_order.id]

Order(id=2, order_type=<OrderType.ASK: 'ask'>, price=100.66665333771964, volume=0, owner_id=10, status=<OrderStatus.FILLED: 3>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), updated=datetime.datetime(2024, 2, 24, 20, 20, 17, 331215), listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008))

In [ ]:
order_list.bisect_left(100)

[Order(id=3, order_type=<OrderType.ASK: 'ask'>, price=100.02901457203792, volume=36, owner_id=6, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008), listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)),
 Order(id=18, order_type=<OrderType.ASK: 'ask'>, price=101.14021182702777, volume=44, owner_id=2, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 188255), updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008), listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)),
 Order(id=8, order_type=<OrderType.ASK: 'ask'>, price=101.65510958678865, volume=52, owner_id=2, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 24, 20, 20, 15, 187257), updated=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008), listed=datetime.datetime(2024, 2, 24, 20, 20, 15, 275008)),
 Order(id=6, order_type=<OrderType.ASK: 'ask'>, price=101.70131585168

### modify tests

In [ ]:
print(len(order_list))
order_list.clear()
order_list

19


SortedList([])

In [ ]:
orders = []
order_list = OrderList(order_type=OrderType.BID)
for i in range(3):
    orders.append(
        Order(
            id=i+1,
            order_type=OrderType.BID,
            price=random.uniform(95, 110),
            volume=random.randint(10, 70),
            owner_id=random.randint(1, 10),
        )
    )
for order in orders:
    order_list.add(order)
print(len(order_list))

for order in order_list:
    print(order)

3
id=1 order_type=<OrderType.BID: 'bid'> price=98.9423934225989 volume=47 owner_id=5 status=<OrderStatus.CREATED: 1> created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659)
id=3 order_type=<OrderType.BID: 'bid'> price=104.28419556843376 volume=18 owner_id=10 status=<OrderStatus.CREATED: 1> created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659)
id=2 order_type=<OrderType.BID: 'bid'> price=107.0168944192405 volume=65 owner_id=9 status=<OrderStatus.CREATED: 1> created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659)


In [ ]:
random_order = random.choice(orders)
print(random_order.id)
order_list.modify(random_order.id, price=100, volume=100)

for order in order_list:
    print(order)

1
id=1 order_type=<OrderType.BID: 'bid'> price=100 volume=100 owner_id=5 status=<OrderStatus.MODIFIED: 4> created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 800212) listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659)
id=3 order_type=<OrderType.BID: 'bid'> price=104.28419556843376 volume=18 owner_id=10 status=<OrderStatus.CREATED: 1> created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659)
id=2 order_type=<OrderType.BID: 'bid'> price=107.0168944192405 volume=65 owner_id=9 status=<OrderStatus.CREATED: 1> created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659) listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659)


Order(id=1, order_type=<OrderType.BID: 'bid'>, price=100, volume=100, owner_id=5, status=<OrderStatus.MODIFIED: 4>, created=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659), updated=datetime.datetime(2024, 2, 24, 20, 20, 19, 800212), listed=datetime.datetime(2024, 2, 24, 20, 20, 19, 32659))

# order book


In [19]:
from datetime import datetime
from typing import List, Optional, Union

# from structures import OrderList, OrderStatus, Order, OrderType


class OrderBook:
    """
    Represents a limit order book that stores and manages buy and sell orders.

    Attributes:
        ask (OrderList): The list of ask orders (sell orders).
        bid (OrderList): The list of bid orders (buy orders).
        order_sources (dict): A dictionary mapping order types to their respective order lists.
        tape (list): A list of filled orders with their details.

    Methods:
        add(order: Order) -> None: Adds a new order to the order book.
        match(order: Order) -> List[Order]: Matches an order with counter orders.
        fill(order: Order, counter_orders: List[Order]) -> None: Fills an order with counter orders.
        match_fill(order: Order) -> None: Matches and fills an order.
        get_order(order: Union[int, Order], order_type: OrderType) -> Optional[Order]: Retrieves an order from the order book.
        search_order(order: Union[Order, int], order_type: Optional[OrderType] = None): Searches for an order in the order book.
        cancel(order: Union[Order, int], order_type: Optional[OrderType] = None): Cancels an order.
        expire(order: Union[Order, int], order_type: Optional[OrderType] = None): Expires an order.
        restore(order: Union[Order, int], order_type: Optional[OrderType] = None): Restores a cancelled or expired order.
        remove_order(order: Union[Order, int], order_type: Optional[OrderType] = None): Removes an order from the order book.
        modify(order: Union[Order, int], order_type: Optional[OrderType] = None, price: Optional[float] = None, volume: Optional[float] = None): Modifies an order's price or volume.
        proceede() -> List[dict]: Retrieves the filled orders from the order book and clears the tape.

    """

    def __init__(self):

        self.ask = OrderList(order_type=OrderType.ASK)
        self.bid = OrderList(order_type=OrderType.BID)
        self.order_sources = {OrderType.ASK: self.ask, OrderType.BID: self.bid}
        self.tape = []

    def add(self, order_: Order) -> None:
        """
        Adds an order to the order book and matches/fills the order if possible.

        :param order_: The order to be added.
        :type order_: Order
        """
        order_.listed = datetime.now()
        self.order_sources[order_.order_type].add(order_)
        self.match_fill(order_)

    def match(self, order_: Order) -> List[Order]:
        """
        Matches the given order with the corresponding orders in the order book.

        :param order_: The order to be matched.
        :type order_: Order
        :return: A list of matched orders.
        :rtype: List[Order]
        """
        matched = []
        if order_.order_type == OrderType.ASK:
            matched = self.bid.bisect_left(order_)
        else:
            matched = self.ask.bisect_right(order_)
        matched.sort(key=lambda x: x.listed)
        return matched

    def fill(self, order_: Order, counter_orders: List[Order]) -> None:
        """
        Fills the given order with the counter orders.

        :param order_: The order to be filled.
        :type order_: Order
        :param counter_orders: The list of counter orders.
        :type counter_orders: List[Order]
        """
        if not counter_orders:
            return
        source = self.order_sources[order_.order_type]
        c_source = self.order_sources[counter_orders[0].order_type]
        for c_order in counter_orders:
            price_ = c_order.price
            volume_ = min(order_.volume, c_order.volume)
            source.fill(order_, volume_)
            c_source.fill(c_order, volume_)
            self.tape.append(
                {
                    "order": order_.id,
                    "contr_order": c_order.id,
                    "price": price_,
                    "volume": volume_,
                    "time": datetime.now(),
                }
            )
            if order_.status == OrderStatus.FILLED:
                break

    def match_fill(self, order: Order) -> None:
        """
        Matches the given order with existing orders in the order book and fills the order if there is a match.

        :param order: The order to be matched and filled.
        :type order: Order
        """
        matched = self.match(order)
        self.fill(order, matched)

    def get_order(
        self, order_: Union[int, Order], order_type_: OrderType
    ) -> Optional[Order]:
        """
        Get an order from the order book.

        :param order_: The order ID or the Order object.
        :type order_: Union[int, Order]
        :param order_type_: The type of the order.
        :type order_type_: OrderType
        :raises ValueError: If the order type is invalid.
        :raises ValueError: If the order is not found.
        :return: The order if found, otherwise None.
        :rtype: Optional[Order]
        """
        order_ = None
        if isinstance(order_, int):
            order_ = self.order_sources[order_type_].get(order_)
        elif isinstance(order_, Order):
            order_ = self.order_sources[order_.order_type].get(order_.id)
        else:
            raise ValueError("Invalid order type")
        if not order_:
            raise ValueError("Order not found")
        return order_

    def search_order(
        self,
        order_: Union[Order, int],
        order_type_: Optional[OrderType] = None,
    ) -> Optional[Order]:
        """
        Search for an order in the order book.

        :param order_: The order to search for. Can be an instance of `Order` or an integer representing the order ID.
        :type order_: Union[Order, int]
        :param order_type_: The type of order to search for. Defaults to None.
        :type order_type_: Optional[OrderType]
        :raises ValueError: If an invalid order type is provided.
        :raises ValueError: If the order is not found in any of the order sources.
        :return: The found order.
        :rtype: Order
        """
        if order_type_:
            return self.get_order(order_, order_type_)
        if isinstance(order_, Order):
            order_type_ = order_.order_type
            return self.get_order(order_, order_type_)
        if isinstance(order_, int):
            order_id = order_
        else:
            raise ValueError("Invalid order type")
        for source in self.order_sources.values():
            order_ = source.get(order_id)
            if order_:
                return order_
        raise ValueError("Order not found")

    def cancel(
        self, order_: Union[Order, int], order_type_: Optional[OrderType] = None
    ) -> None:
        """
        Cancels the specified order.

        :param order_: The order to be cancelled.
        :type order_: Union[Order, int]
        :param order_type_: The type of the order to be cancelled, defaults to None.
        :type order_type_: Optional[OrderType], optional
        :raises ValueError: If the order is already cancelled or expired.
        """
        order_ = self.search_order(order_, order_type_)
        if not order_.status.is_active:
            raise ValueError("Order is already cancelled or expired")
        source = self.order_sources[order_.order_type]
        source.cancel(order_)

    def expire(
        self, order_: Union[Order, int], order_type_: Optional[OrderType] = None
    ) -> None:
        """
        Expire the given order.

        :param order_: The order to expire.
        :type order_: Union[Order, int]
        :param order_type_: The type of the order to expire, defaults to None.
        :type order_type_: Optional[OrderType], optional
        :raises ValueError: If the order is already cancelled or expired.
        """
        order_ = self.search_order(order_, order_type_)
        if not order_.status.is_active:
            raise ValueError("Order is already cancelled or expired")
        source = self.order_sources[order_.order_type]
        source.expire(order_)

    def restore(
        self, order_: Union[Order, int], order_type_: Optional[OrderType] = None
    ) -> None:
        """
        Restores an order to the order book.

        :param order_: The order or order ID to be restored.
        :type order_: Union[Order, int]
        :param order_type_: The type of the order to be restored, defaults to None.
        :type order_type_: Optional[OrderType], optional
        :raises ValueError: If the order is already active.
        """

        order_ = self.search_order(order_, order_type_)
        if order_.status.is_active:
            raise ValueError("Order is already active")
        source = self.order_sources[order_.order_type]
        source.relist(order_)
        self.match_fill(order_)

    def remove_order(
        self, order_: Union[Order, int], order_type_: Optional[OrderType] = None
    ) -> None:
        """
        Remove an order from the order book.

        :param order_: The order to be removed. Can be an instance of `Order` or the order ID (int).
        :type order_: Union[Order, int]
        :param order_type_: The type of order to be removed. Defaults to None.
        :type order_type_: Optional[OrderType]
        """
        order_ = self.search_order(order_, order_type_)
        source = self.order_sources[order_.order_type]
        source.remove(order_)

    def modify(
        self,
        order_: Union[Order, int],
        order_type_: Optional[OrderType] = None,
        price_: Optional[float] = None,
        volume_: Optional[float] = None,
    ) -> None:
        """
        Modifies an existing order in the order book.

        :param order_: The order object or order ID to be modified.
        :type order_: Union[Order, int]
        :param order_type_: The new order type (buy/sell), defaults to None.
        :type order_type_: Optional[OrderType], optional
        :param price_: The new price of the order, defaults to None.
        :type price_: Optional[float], optional
        :param volume_: The new volume of the order, defaults to None.
        :type volume_: Optional[float], optional
        """
        order_ = self.search_order(order_, order_type_)
        source = self.order_sources[order_.order_type]
        source.modify(order_, price_, volume_)
        self.match_fill(order_)

    def proceede(self):

        tape = self.tape.copy()
        self.tape.clear()
        return tape

### little tests

In [20]:
order_book = OrderBook()
i_gen = OrderIdGenerator()
next(i_gen)

300

In [21]:
for i in range(random.randint(10, 25)):
    order_book.add(
        Order(
            id=next(i_gen),
            order_type=OrderType.ASK,
            price=random.uniform(95, 130),
            volume=random.randint(10, 70),
            owner_id=random.randint(1, 10),
        )
    )
for i in range(random.randint(10, 25)):
    order_book.add(
        Order(
            id=next(i_gen),
            order_type=OrderType.BID,
            price=random.uniform(70, 110),
            volume=random.randint(10, 70),
            owner_id=random.randint(1, 10),
        )
    )


In [22]:
order_book.tape

[{'order': 318,
  'contr_order': 306,
  'price': 105.37243413682314,
  'volume': 16,
  'time': datetime.datetime(2024, 2, 25, 17, 59, 18, 362434)},
 {'order': 319,
  'contr_order': 306,
  'price': 105.37243413682314,
  'volume': 17,
  'time': datetime.datetime(2024, 2, 25, 17, 59, 18, 362434)},
 {'order': 321,
  'contr_order': 313,
  'price': 100.03902443350304,
  'volume': 45,
  'time': datetime.datetime(2024, 2, 25, 17, 59, 18, 363429)},
 {'order': 327,
  'contr_order': 313,
  'price': 100.03902443350304,
  'volume': 11,
  'time': datetime.datetime(2024, 2, 25, 17, 59, 18, 363429)}]

In [23]:
order_book.search_order(327)

Order(id=327, order_type=<OrderType.BID: 'bid'>, price=101.88389998612206, volume=43, owner_id=3, status=<OrderStatus.PARTIALLY_FILLED: 2>, created=datetime.datetime(2024, 2, 25, 17, 59, 18, 363429), updated=datetime.datetime(2024, 2, 25, 17, 59, 18, 363429), listed=datetime.datetime(2024, 2, 25, 17, 59, 18, 363429))

In [24]:
order_book.cancel(327)
order_book.search_order(327)

Order(id=327, order_type=<OrderType.BID: 'bid'>, price=101.88389998612206, volume=43, owner_id=3, status=<OrderStatus.CANCELLED: 5>, created=datetime.datetime(2024, 2, 25, 17, 59, 18, 363429), updated=datetime.datetime(2024, 2, 25, 17, 59, 44, 490092), listed=datetime.datetime(2024, 2, 25, 17, 59, 18, 363429))

In [25]:
order_book.cancel(327)
# error

ValueError: Order is already cancelled or expired

In [26]:
order_book.restore(327)
order_book.search_order(327)

Order(id=327, order_type=<OrderType.BID: 'bid'>, price=101.88389998612206, volume=43, owner_id=3, status=<OrderStatus.RESTORED: 6>, created=datetime.datetime(2024, 2, 25, 17, 59, 18, 363429), updated=datetime.datetime(2024, 2, 25, 18, 1, 38, 635563), listed=datetime.datetime(2024, 2, 25, 18, 1, 38, 635563))

In [27]:
order_book.restore(327)
# error

ValueError: Order is already active

In [28]:
order_book.expire(327)

In [29]:
order_book.ask

SortedList([Order(id=306, order_type=<OrderType.ASK: 'ask'>, price=105.37243413682314, volume=36, owner_id=6, status=<OrderStatus.PARTIALLY_FILLED: 2>, created=datetime.datetime(2024, 2, 25, 17, 59, 18, 361436), updated=datetime.datetime(2024, 2, 25, 17, 59, 18, 362434), listed=datetime.datetime(2024, 2, 25, 17, 59, 18, 361436)), Order(id=311, order_type=<OrderType.ASK: 'ask'>, price=109.0295845890462, volume=56, owner_id=8, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 25, 17, 59, 18, 362434), updated=datetime.datetime(2024, 2, 25, 17, 59, 18, 362434), listed=datetime.datetime(2024, 2, 25, 17, 59, 18, 362434)), Order(id=302, order_type=<OrderType.ASK: 'ask'>, price=110.27285432828684, volume=13, owner_id=8, status=<OrderStatus.CREATED: 1>, created=datetime.datetime(2024, 2, 25, 17, 59, 18, 361436), updated=datetime.datetime(2024, 2, 25, 17, 59, 18, 361436), listed=datetime.datetime(2024, 2, 25, 17, 59, 18, 361436)), Order(id=310, order_type=<OrderType.ASK: 'ask'>

In [31]:
order_book.modify(306, price_=100, volume_=10)
order_book.search_order(306)

Order(id=306, order_type=<OrderType.ASK: 'ask'>, price=100, volume=10, owner_id=6, status=<OrderStatus.MODIFIED: 4>, created=datetime.datetime(2024, 2, 25, 17, 59, 18, 361436), updated=datetime.datetime(2024, 2, 25, 18, 3, 17, 647134), listed=datetime.datetime(2024, 2, 25, 17, 59, 18, 361436))

In [32]:
order_book.modify(306, price_=70, volume_=100)
order_book.search_order(306)

Order(id=306, order_type=<OrderType.ASK: 'ask'>, price=70, volume=0, owner_id=6, status=<OrderStatus.FILLED: 3>, created=datetime.datetime(2024, 2, 25, 17, 59, 18, 361436), updated=datetime.datetime(2024, 2, 25, 18, 4, 30, 812469), listed=datetime.datetime(2024, 2, 25, 17, 59, 18, 361436))

In [33]:
order_book.tape

[{'order': 318,
  'contr_order': 306,
  'price': 105.37243413682314,
  'volume': 16,
  'time': datetime.datetime(2024, 2, 25, 17, 59, 18, 362434)},
 {'order': 319,
  'contr_order': 306,
  'price': 105.37243413682314,
  'volume': 17,
  'time': datetime.datetime(2024, 2, 25, 17, 59, 18, 362434)},
 {'order': 321,
  'contr_order': 313,
  'price': 100.03902443350304,
  'volume': 45,
  'time': datetime.datetime(2024, 2, 25, 17, 59, 18, 363429)},
 {'order': 327,
  'contr_order': 313,
  'price': 100.03902443350304,
  'volume': 11,
  'time': datetime.datetime(2024, 2, 25, 17, 59, 18, 363429)},
 {'order': 306,
  'contr_order': 333,
  'price': 103.18198378434235,
  'volume': 12,
  'time': datetime.datetime(2024, 2, 25, 18, 3, 0, 253009)},
 {'order': 306,
  'contr_order': 329,
  'price': 104.97470393213263,
  'volume': 50,
  'time': datetime.datetime(2024, 2, 25, 18, 3, 0, 253009)},
 {'order': 306,
  'contr_order': 317,
  'price': 70.52826489482837,
  'volume': 35,
  'time': datetime.datetime(2024